# Decision Trees

**Qué vamos a ver:**
- Ajuste de hiperparámetros en los árboles de decisión
- Combinar el ajuste de hiperparámetros y la evaluación de modelos
    - GridSearch
    - RandomSearch
    - Optimización basada en modelos

## Hiperparámetros de los árboles de decisión. Ajuste de los árboles de decisión.
---
- *max_depth : int or None, opcional (por defecto=None)*

    La profundidad máxima del árbol. Si es None, los nodos se expanden hasta que todas las hojas sean puras o hasta que todas las hojas contengan menos muestras que min_samples_split. Se ignora si max_leaf_nodes no es None.
    
- *min_samples_split : int, opcional (por defecto=2)*

    El número mínimo de muestras necesarias para dividir un nodo interno.

- Hay más parámetros:
  - https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
  - https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [1]:
# Cargan los datos, las entradas van a X, las salidas a y.
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_california_housing
from scipy.stats import sem

housing_meta = fetch_california_housing()
X = housing_meta.data
y = housing_meta.target

print(housing_meta.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

## Combinación entre el ajuste de hiperparámetros y la evaluación del modelo.
---
La combinación de evaluación de modelos y ajuste de hiperparámetros puede entenderse como un bucle externo (*outer*) que entrena un modelo y prueba el modelo, y un bucle interno (*inner*), donde el proceso de entrenamiento consiste en buscar los mejores hiperparámetros, y luego obtener el modelo con esos mejores hiperparámetros.

- En primer lugar, vamos a utilizar **Holdout** (entrenar/probar) para la evaluación del modelo (bucle externo o *outer*)
- **3-fold crossvalidation** para el ajuste de los hiperparámetros (bucle interno o *inner*)
- Los hiperparámetros se ajustarán con **Gridsearch**

### <u>1. GRIDSEARCH.</u>

Tabla en la que se prueba todo con todo --> busqueda

**Función RMSE**

In [2]:
# Importamos las librerías necesarias.
import numpy as np
from sklearn import metrics

# Definamos nuestra función python para el RMSE.
def rmse(y_test, y_test_pred):
  """ This is my computation of Root Mean Squared Error """
  return np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

In [3]:
# Importamos la función train_test_split de sklearn
from sklearn.model_selection import train_test_split

# Holdout para el modelo de evaluación. 33% of available data for test
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.33, random_state=42)

**RMSE con hiperparámetros por defecto**

En primer lugar, recordemos el RMSE con hiperparámetros por defecto

In [4]:
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor

regr = DecisionTreeRegressor()
np.random.seed(42)
regr.fit(X=X_train, y=y_train)
print(f"RMSE of tree with default hyper-pars: {rmse(y_test, regr.predict(X=X_test))}")

RMSE of tree with default hyper-pars: 0.7389833060012664


In [5]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import KFold


# Search space
param_grid = {'max_depth': list(range(2,16,2)),
              'min_samples_split': list(range(2,16,2))}

inner = KFold(n_splits=3, shuffle=True, random_state=42)

# Definition of a 2-step process that self-adjusts 2 hyperpars
regr = GridSearchCV(DecisionTreeRegressor(),
                   param_grid,
                   scoring='neg_mean_squared_error',
                   cv=inner,
                   n_jobs=1, verbose=1)

# Train the self-adjusting process
np.random.seed(42)
regr.fit(X=X_train, y=y_train)

# At this point, regr contains the model with the best hyper-parameters found by gridsearch
# and trained on the complete X_train

Fitting 3 folds for each of 49 candidates, totalling 147 fits


GridSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
             estimator=DecisionTreeRegressor(), n_jobs=1,
             param_grid={'max_depth': [2, 4, 6, 8, 10, 12, 14],
                         'min_samples_split': [2, 4, 6, 8, 10, 12, 14]},
             scoring='neg_mean_squared_error', verbose=1)

In [6]:
# Now, the performance of regr is computed on the test partition
print(f"RMSE of tree with hyper-parameter tuning (grid-search): {rmse(y_test, regr.predict(X=X_test))}")

RMSE of tree with hyper-parameter tuning (grid-search): 0.6647688056188821


In [7]:
# Veamos los mejores hiperparámetros y su puntuación (MSE). 
# Podemos ver que están cerca de los extremos del espacio de parámetros, pero no en el extremo.
regr.best_params_, -regr.best_score_

({'max_depth': 8, 'min_samples_split': 14}, 0.44782374157784616)

### <u>2. Búsqueda aleatoria </u>(*RANDOMIZED SEARCH*)

Ahora, utilicemos *Randomized Search* en lugar de *GridSearch*. 

No es la mejor solución.

**3-KFold**

In [8]:
# Sólo se probarán 20 combinaciones de valores de hiperparámetros (budget=20)

from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn import metrics

# Search space
param_grid = {'max_depth': list(range(2,16,2)),
              'min_samples_split': list(range(2,16,2))}

# Inner evaluation
inner = KFold(n_splits=3, shuffle=True, random_state=42)

# Cambia esto: ahora busca en budget 20 y en random
budget = 20
regr = RandomizedSearchCV(DecisionTreeRegressor(),
                         param_grid,
                         scoring='neg_mean_squared_error',
                         cv=inner,
                         n_jobs=1, verbose=1,
                         n_iter=budget
                        )
np.random.seed(42)
regr.fit(X=X_train, y=y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
                   estimator=DecisionTreeRegressor(), n_iter=20, n_jobs=1,
                   param_distributions={'max_depth': [2, 4, 6, 8, 10, 12, 14],
                                        'min_samples_split': [2, 4, 6, 8, 10,
                                                              12, 14]},
                   scoring='neg_mean_squared_error', verbose=1)

In [9]:
# Now, the performance of regr is computed on the test partition
print(f"RMSE of tree with hyper-parameter tuning (random search): {rmse(y_test, regr.predict(X=X_test))}")

RMSE of tree with hyper-parameter tuning (random search): 0.6640044054767447


**Resultados**

El valor de error empeora.

Posibles mejores soluciones:
- Busqueda bayesiana
- Buqueda algoritmo genético

In [10]:
regr.best_params_, -regr.best_score_

({'min_samples_split': 12, 'max_depth': 8}, 0.4452664287620658)

**Unifor y expon**

Para la **Búsqueda Aleatoria**, podemos definir el espacio de búsqueda con distribuciones estadísticas, en lugar de utilizar valores particulares como hacíamos antes. A continuación puedes ver cómo utilizar una distribución uniforme sobre enteros entre 2 y 16 mediante *randint*. Para hiperparámetros continuos podríamos usar distribuciones continuas como *uniform* o *expon* (exponencial).

In [11]:
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn import metrics
from scipy.stats import uniform, expon
from scipy.stats import randint as sp_randint

# Search space with integer uniform distributions
param_grid = {'max_depth': sp_randint(2,16),
              'min_samples_split': sp_randint(2,16)}

inner = KFold(n_splits=3, shuffle=True, random_state=42)

budget = 20
regr = RandomizedSearchCV(DecisionTreeRegressor(),
                         param_grid,
                         scoring='neg_mean_squared_error',
                         cv=inner,
                         n_jobs=1, verbose=1,
                         n_iter=budget
                        )

np.random.seed(42)
regr.fit(X=X_train, y=y_train)

# At this point, regr contains the model with the best hyper-parameters found by gridsearch
# and trained on the complete X_train

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
                   estimator=DecisionTreeRegressor(), n_iter=20, n_jobs=1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x17d1b3220>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x17d19ef70>},
                   scoring='neg_mean_squared_error', verbose=1)

In [12]:
# Now, the performance of regr is computed on the test partition
print(f"RMSE of tree with hyper-parameter tuning (random search II): {rmse(y_test, regr.predict(X=X_test))}")

RMSE of tree with hyper-parameter tuning (random search II): 0.6640044054767447


In [13]:
regr.best_params_, -regr.best_score_

({'max_depth': 8, 'min_samples_split': 12}, 0.4483635388476918)

**5-KFold**

¿Y si quisiéramos hacer **evaluación de modelos con 5-fold crossvalidation** y **ajuste de hiperparámetros con 3-fold crossvalidation**? Esto se denomina validación cruzada anidada (https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html). Hay un bucle externo (para evaluar los modelos) y un bucle interno (para ajustar los hiperparámetros)

In [14]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn import metrics

# random_state=42 for reproducibility
# Evaluation of model (outer loop)
outer = KFold(n_splits=5, shuffle=True, random_state=42)


from scipy.stats import uniform, expon

# Search space
param_grid = {'max_depth': list(range(2,16,2)),
              'min_samples_split': list(range(2,16,2))}

inner = KFold(n_splits=3, shuffle=True, random_state=42)

budget = 20
# This is the internal 3-fold crossvalidation for hyper-parameter tuning
regr = RandomizedSearchCV(DecisionTreeRegressor(),
                         param_grid,
                         scoring='neg_mean_squared_error',
                         # 3-fold for hyper-parameter tuning
                         cv=inner,
                         n_jobs=1, verbose=1,
                         n_iter=budget
                        )

# This is the external 5-fold crossvalidation for model evaluation
# Notice that regr is the model resulting of hyper-parameter tuning
np.random.seed(42)

# For sklearn, higher scores are better. Given that MSE is an error (smaller is better), the corresponding score is -MSE
scores = -cross_val_score(regr,
                            X, y,
                            scoring='neg_mean_squared_error',
                            cv = outer)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Fitting 3 folds for each of 20 candidates, totalling 60 fits


**Resultados**

In [15]:
print(scores)
# The score was MSE, we want RMSE
scores = np.sqrt(scores)
# The mean of the 5-fold crossvalidation is the final score of the model
print(f"{scores.mean()} +- {scores.std()}")

[0.42013123 0.40583334 0.39388899 0.4006011  0.41832784]
0.6385089628008911 +- 0.00792487477201872


**Modelo final**

Obtención del modelo final (para su despliegue, o para enviarlo a un concurso, ...)

Si necesitamos un modelo final, podemos obtenerlo ajustando regr a todos los datos disponibles. Recordemos que regr hace ajuste de hiper-parámetros.

In [16]:
np.random.seed(42)

# Fitting again the randomized search HPO
regrFinal = regr.fit(X,y)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


In [17]:
regr.best_params_, -regr.best_score_

({'min_samples_split': 14, 'max_depth': 10}, 0.3993552378121243)

### <u>3. Optimización basada en modelos.</u> (*BAYESIAN OPTIMIZATION*)

Para ello se utilizará scikit-optimize (skopt): https://scikit-optimize.github.io. **Holdout** para la evaluación del modelo y **3-fold crossvalidation** para el ajuste de hiperparámetros (con **Model Based Optimization** ).

In [18]:
#!pip install scikit-learn==0.24.2
#!pip install scikit-optimize
#!pip install git+https://github.com/scikit-optimize/scikit-optimize.git

# Checking that everything is correct with skopt (0.9.dev0) and sklearn

from skopt import __version__
print(__version__)
from sklearn import __version__
print(__version__)


0.9.0
1.3.2


In [2]:
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical
from sklearn import metrics
from skopt.plots import plot_objective, plot_histogram, plot_convergence
from sklearn.model_selection import KFold

# Search space with integer uniform distributions
param_grid = {'max_depth': Integer(2,20),
              'min_samples_split': Integer(20,100)}

inner = KFold(n_splits=3, shuffle=True, random_state=42)

# Cambiamos otra vez el budget y el tipo de busqueda
budget = 20
regr = BayesSearchCV(DecisionTreeRegressor(),
                    param_grid,
                    scoring='neg_mean_squared_error',
                    cv=inner,
                    n_jobs=1, verbose=1,
                    n_iter=budget
                    )
np.random.seed(42)
regr.fit(X=X_train, y=y_train)

# At this point, regr contains the model with the best hyper-parameters found by bayessearch
# and trained on the complete X_train

NameError: name 'DecisionTreeRegressor' is not defined

In [21]:
# Now, the performance of regr is computed on the test partition
print(f"RMSE of tree with hyper-parameter tuning (bayesian optimization): {rmse(y_test, regr.predict(X=X_test))}")

AttributeError: 'BayesSearchCV' object has no attribute 'best_estimator_'

In [22]:
regr.best_params_, -regr.best_score_

AttributeError: 'BayesSearchCV' object has no attribute 'best_params_'

**Plot de convergencia**

In [ ]:
# Podemos comprobar si la optimización ha convergido
_ = plot_convergence(regr.optimizer_results_[0])
plt.show()

In [ ]:
# Deberíamos de llegar a parar en una recta.
_ = plot_objective(regr.optimizer_results_[0],
                   dimensions=['max_depth', 'min_samples_split'],
                   n_minimum_search=int(1e8))
plt.show()

**Importante**

El número de ejemplos mínimo --> 48.

Para la práctica: random y si queremos el bayesiana (pros)